# Inference demo for Cervical Cancer subclass classifier
In this notebook, we demonstrate how to load a pre-trained Cervical Cancer Subclass Classifier and use it to predict the class of new images. The notebook will guide you through the process of loading the model, making predictions, and visualizing the results.

In [ ]:
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.models.cervical_cancer_model import CervicalModel

In [ ]:
# Check if CUDA is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Config settings
num_classes = 5

# Load model with checkpoint
model = CervicalModel(num_classes=num_classes)
checkpoint_path = '../results/cervical_checkpoints/best_checkpoint.pth'  # Your path
model.load_state_dict(torch.load(checkpoint_path, map_location=device, weights_only=True))
model.to(device)
model.eval()
print("Model loaded successfully!")

In [ ]:
# Visualize image
def imshow(img):
    """Display an image from a tensor."""
    img = img / 2 + 0.5 # Undo normalization
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Image prediction
def predict_image(model, image_path, class_names):
    """Predict the class of an image using the trained model."""
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Model is trained with 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize as in training
    ])
    
    image = Image.open(image_path)
    plt.imshow(image)  # Show original iamge
    plt.title("Original Image")
    plt.show()

    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    predicted_class = class_names[predicted.item()]
    print(f"Predicted Class: {predicted_class}")
    return predicted_class


In [ ]:
import random

# Path to test dataset
test_images_path = 'D:/Data/cervical_cancer_data/test/'

# Get random image
category = random.choice(os.listdir(test_images_path))
image_path = os.path.join(test_images_path, category, random.choice(os.listdir(os.path.join(test_images_path, category))))
print(f"Selected Image: {image_path}")

# Class names list
class_names = os.listdir(test_images_path)

# Do prediction
predicted_class = predict_image(model, image_path, class_names)


In [ ]:
# Show multiple images with their predictions
def visualize_predictions(model, test_images_path, class_names, num_images=5):
    """Visualize predictions for multiple random images from the test set."""
    plt.figure(figsize=(15, 10))
    for i in range(num_images):
        category = random.choice(os.listdir(test_images_path))
        image_path = os.path.join(test_images_path, category, random.choice(os.listdir(os.path.join(test_images_path, category))))
        
        image = Image.open(image_path)
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        image_tensor = transform(image).unsqueeze(0).to(device)
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_class = class_names[predicted.item()]
        
        plt.subplot(1, num_images, i + 1)
        plt.imshow(image)
        plt.title(f"Predicted: {predicted_class}", color="green")
        plt.axis('off')
    
    plt.show()

# Show 5 predictions
visualize_predictions(model, test_images_path, class_names, num_images=5)

# Conclusion
In this notebook, we have demonstrated how to load a pre-trained Cervical Cancer Subclass Classifier, predict the subclass of new images, and visualize the model's predictions. This notebook provides an intuitive way to test the performance of the model on unseen data. For a more detailed evaluation, refer to the training and test reports in the `results/` folder.